In [13]:
import pandas as pd
import os

from datetime import datetime, timedelta

import eikon as ek

## Import Data

Import to remember to set your api_key in the code

In [14]:
# Creating Class to Import Data

class ImportData:
 
    def __init__(self, filename = None):
        
        # Constants as instance attributes
        self.end_date = datetime.today()
        self.start_date = self.end_date - timedelta(days=7*365)
        self.filename = filename

        self.export_folder = "../../data/processed"
        self.raw_folder = "../../data/raw"

        os.makedirs(self.export_folder, exist_ok=True)
    
    # Import Data

    def import_eikon_series(self, ticker_code: str, app_key):
        
        # Ensure valid API key
        ek.set_app_key(app_key)  # Make sure to use a valid API key
        
        # Fetch data from Eikon
        df = ek.get_timeseries([ticker_code], 
                               start_date=self.start_date, 
                               end_date=self.end_date)

        # Reset index to make the date a column
        df.reset_index(inplace=True)

        # Rename columns for clarity
        df.columns = ['Date', ticker_code]

        filepath = os.path.join(self.export_folder, f"{self.filename}.csv")
        df.to_csv(filepath, index=False)
        print(f"Data exported to {filepath}")
    
        return df
    
    def import_eia_raw_data(self, sheet_name, col_position):

        # Listar arquivos disponíveis no diretório
        files = [f for f in os.listdir(self.raw_folder) if f.lower().endswith(('.csv', '.xls', '.xlsx')) and f != '.DS_Store']
    
        if not files:
            raise FileNotFoundError(f"Nenhum arquivo encontrado em {self.raw_folder}")

        # Seleciona o primeiro arquivo encontrado (ajuste conforme necessário)
        file_import_name = files[0]  
        file_path_import = os.path.join(self.raw_folder, file_import_name)

        # Importar o arquivo Excel
        df = pd.read_excel(file_path_import, sheet_name=sheet_name, usecols=col_position, skiprows= 2)

        filepath = os.path.join(self.export_folder, f"{self.filename}.csv")
        df.to_csv(filepath, index=False)
        print(f"Data exported to {filepath}")

        return df
        


In [15]:
# Import Crude Oil Data
ImportData("us_endingcrudeoil_stocks").import_eia_raw_data("Data 1", [0, 3])
ImportData("us_endingcommercialcrudeoil_stocks").import_eia_raw_data("Data 1", [0, 4])

# Refined Products
ImportData("us_endinggasoline_stocks").import_eia_raw_data("Data 1", [0, 8])
ImportData("us_endingdistillates_stocks").import_eia_raw_data("Data 1", [0, 26])

Data exported to ../../data/processed/us_endingcrudeoil_stocks.csv
Data exported to ../../data/processed/us_endingcommercialcrudeoil_stocks.csv
Data exported to ../../data/processed/us_endinggasoline_stocks.csv
Data exported to ../../data/processed/us_endingdistillates_stocks.csv


,Date,Weekly U.S. Ending Stocks of Distillate Fuel Oil (Thousand Barrels)
0,1982-08-20,149415
1,1982-08-27,154589
2,1982-09-24,158684
3,1982-10-01,154461
4,1982-10-08,158242
...,...,...
2209,2025-01-31,118480
2210,2025-02-07,118615
2211,2025-02-14,116564
2212,2025-02-21,120472
